Correlated Subquery Join of Tables is simple in SQL.
It is not so intuitive, as it turned out for python pandas.
Following is an attempt to do the correlated subquery merge of df into ts, in which df is a rank of IDs with sporadic eff_dt, while ts is a list of values for all dates.

In [169]:
import pandas as pd
import numpy as np
import random

In [174]:
df = pd.DataFrame({'eff_dt' : random.sample(
            pd.date_range('2000-1-1',periods=5, freq='2D').tolist(),4),
              'id' : ['A','A','B','B'],
                'rank' : random.sample(range(30), 4)})
df.sort_values(by=['id','eff_dt'],inplace=True)
df

,eff_dt,id,rank
0,2000-01-01,A,7
1,2000-01-05,A,27
3,2000-01-03,B,3
2,2000-01-09,B,18


Given: df is a rank of of id with sparse eff_dt, created randomly like the above

In [175]:
n = 7
ts = pd.concat([pd.DataFrame({'dt' : pd.date_range('2000-1-1',periods=n, freq='D').tolist(),
                    'id' : ['A'] * n,
                    'val' : random.sample(range(20), n)}),
               pd.DataFrame({'dt' : pd.date_range('2000-1-1',periods=n, freq='D').tolist(),
                    'id' : ['B'] * n,
                    'val' : random.sample(range(20), n)})
      ])
ts.sort_values(by=['id','dt'],inplace=True)
ts

,dt,id,val
0,2000-01-01,A,1
1,2000-01-02,A,4
2,2000-01-03,A,0
3,2000-01-04,A,12
4,2000-01-05,A,15
5,2000-01-06,A,2
6,2000-01-07,A,17
0,2000-01-01,B,14
1,2000-01-02,B,11
2,2000-01-03,B,16


Given: ts is a list of values for ID: [A,B] between ['2000-01-01' and '2000-01-07']
Objective: merge df into ts, with the corresponding rank for given ts.dt

First: modify df's column structure from eff_dt with start_dt and end_dt.
end_dt being next eff_dt for given ID.

In [176]:
df['end'] = df.groupby(['id'])['eff_dt'].shift(-1)
df.rename(columns={'eff_dt':'start'},inplace=True)

In [177]:
# Left join ts, with df by id.
# Note: this is a cartesian product
t=ts.merge(df, on='id', how='left').sort_values(by=['id','dt'])
# invalidate rank, when rank's start_dt > id's dt.
t['rank'] = t.apply(lambda r: np.NaN if r['start'] > r['dt'] else r['rank'], axis = 1 )
# filter out only records that is of interest, since above left join created a cartesian product of dates
t=t.query("(start <= dt and end > dt) or  (start < dt and end == 'NaT') or(start >= dt) ")
# find unique columns
t=t.groupby(['dt','id','val']).first().reset_index()
# sort and drop start,end
t.sort_values(by=['id','dt'],inplace=True)
t.drop(['start','end'],axis=1,inplace=True)
t

,dt,id,val,rank
0,2000-01-01,A,1,7.0
2,2000-01-02,A,4,7.0
4,2000-01-03,A,0,7.0
6,2000-01-04,A,12,7.0
8,2000-01-05,A,15,27.0
10,2000-01-06,A,2,27.0
12,2000-01-07,A,17,27.0
1,2000-01-01,B,14,NaN
3,2000-01-02,B,11,NaN
5,2000-01-03,B,16,3.0
